In [1]:
%%capture
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline

In [2]:
%%capture
import sys
sys.path.append('/home/wrwt/Programming/pygraphmodels')
import graphmodels as gm

In [3]:
import warnings
warnings.filterwarnings('ignore', module='matplotlib')
warnings.filterwarnings('ignore', module='IPython')

In [4]:
def bic_score(x, pa):
    l = gm.information.discrete_mutual_information(data[[x]], data[pa]) - gm.information.discrete_entropy(data[[x]])
    n = data.shape[0]
    k = np.prod(true_dgm.factor(x).table.shape) - 1
    result = -n*l + 0.5 * np.log(n) * k
    return result

In [5]:
from os import listdir
import os.path
NETWORKS_PATH = '/home/wrwt/Programming/pygraphmodels/networks/'
network_filenames = listdir(NETWORKS_PATH)
true_dgm = gm.DGM.read(os.path.join(NETWORKS_PATH, 'earthquake.bif'))
true_dgm.draw()

In [6]:
data = true_dgm.rvs(100000)

In [7]:
true_dgm.remove_edge('Burglary', 'Alarm')
true_dgm.add_edge('Alarm', 'Burglary')
true_dgm.add_edge('Burglary', 'MaryCalls')

In [8]:
from graphmodels.information import discrete_entropy, discrete_mutual_information

In [9]:
from graphmodels import ScoreBIC, MatrixGraph, InformationEstimator, EntropyEstimator
mxg = MatrixGraph.from_networkx_DiGraph(true_dgm, order=data.columns)
ie = InformationEstimator(mxg, data)
ee = EntropyEstimator(mxg, data)
mxg.names_to_idx

{'Alarm': 2, 'Burglary': 1, 'Earthquake': 0, 'JohnCalls': 4, 'MaryCalls': 3}

In [10]:
discrete_mutual_information(data[['Alarm']], data[['Burglary']])

0.039202909914051148

In [11]:
ie(2, [0, 1, 0, 0, 0])

0.039202901348827923

In [12]:
discrete_entropy(data[['Burglary']])

0.054201659512216296

In [13]:
discrete_mutual_information(data[['Alarm']], data[['Burglary', 'Earthquake']])

0.058825984667598349

In [14]:
discrete_mutual_information(data[['Alarm']], data[['Earthquake']])

0.013407474257951692

In [15]:
discrete_mutual_information(data[['Burglary']], data[['Earthquake']])

1.9743910894341635e-05

In [16]:
def bic_score(dgm, data):
    result = 0.
    for x in dgm.nodes():
        pa = dgm.predecessors(x)
        
        def n_values(x):
            return len(data[x].value_counts())
    
        k = n_values(x)*np.prod([n_values(pa_i) for pa_i in pa]) - 1
        n = data.shape[0]
        l = n*(discrete_mutual_information(data[[x]], data[pa]))
        result += l - 0.5 * np.log(n) * k
    return result

In [17]:
    def __call__(self, node, parents):
        parents = np.asarray(parents, dtype=bool)
        k = self.n_values[node]*np.prod(self.n_values[parents]) - 1
        n = self.data.shape[0]
        l = n*self.mi_estimator(node, parents)

        result = l - 0.5 * np.log(n) * k
        return result

In [18]:
from itertools import permutations
dgm = gm.DGM(true_dgm)

scores = []
ops = []
for src, dst in permutations(dgm.nodes(), 2):
    if dgm.has_edge(src, dst):
        dgm.remove_edge(src, dst)
        if nx.is_directed_acyclic_graph(dgm):
            scores.append(bic_score(dgm, data))
            ops.append(('remove', src, dst))
        dgm.add_edge(dst, src)
        if nx.is_directed_acyclic_graph(dgm):
            scores.append(bic_score(dgm, data))
            ops.append(('reverse', src, dst))
        dgm.remove_edge(dst, src)
        dgm.add_edge(src, dst)
    else:
        dgm.add_edge(src, dst)
        if nx.is_directed_acyclic_graph(dgm):
            scores.append(bic_score(dgm, data))
            ops.append(('add', src, dst))
        dgm.remove_edge(src, dst)

In [19]:
bic_score(dgm, data)

12540.979345890733

In [20]:
bic = ScoreBIC(mxg, data)
bic.total()

12540.974279539529

In [ ]:
max(scores)

Как видно, правильная сеть является локальным максимумом BIC score.

In [ ]:
ops[np.argmax(scores)]